In [1]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict

In [2]:
availability = pd.read_csv('/home/farnoosh/git_repos/hymate_task/input_data/availability.csv')

prices = pd.read_csv('/home/farnoosh/git_repos/hymate_task/input_data/prices.csv')

with open('/home/farnoosh/git_repos/hymate_task/input_data/cars.json') as f:
    cars = json.load(f)

with open('/home/farnoosh/git_repos/hymate_task/input_data/requirements.json') as f:
    requirements = json.load(f)

In [3]:
CAR_IDS = ['Car_1', 'Car_2', 'Car_3', 'Car_4', 'Car_5']
DAYS = ['0', '1', '2', '3', '4', '5']
MAX_PORTS = 3
EFFICIENCY = 0.95

In [4]:
def get_hour_index(day, hour):
    return int(day) * 24 + hour

# Calculate cost of charging for a car
def calculate_cost(car_id, day, hour, initial_soc, availability_df, prices_df, cars_list, efficiency=0.95):
    hour_idx = get_hour_index(day, hour)
    car = next(car for car in cars_list if car['id'] == car_id)

    # If there is no requirement data for a car_id, do nothing (for car_5)
    try:
        target_soc = next(iter(requirements[car_id][day].values()))
    except KeyError:
        return None

    # Calculate the total daily energy and cost
    delta_soc = target_soc - initial_soc
    if delta_soc <= 0 or availability_df.loc[hour_idx, car_id] != 1:
        return None

    energy_needed = delta_soc * car['capacity_kWh']
    real_energy_needed = energy_needed / efficiency
    hours_needed = int(np.ceil(real_energy_needed / car['max_charge_rate_kW']))

    total_energy_charged, total_cost = 0, 0
    charging_schedule = {}

    for h in range(hour_idx, hour_idx + hours_needed):
        if h >= len(prices_df) or availability_df.loc[h, car_id] != 1:
            return None
        price = prices_df.loc[h, 'price_eur_per_kWh']
        energy_this_hour = min(car['max_charge_rate_kW'], (real_energy_needed - total_energy_charged))
        total_energy_charged += energy_this_hour
        total_cost += energy_this_hour * price
        charging_schedule[h] = round(energy_this_hour, 2)
        if total_energy_charged >= real_energy_needed:
            break

    finish_hour = max(charging_schedule.keys()) if charging_schedule else None
    return {
        'car_id': car_id,
        'day': day,
        'start_hour': hour_idx,
        'finish_hour': finish_hour,
        'hours_charging': len(charging_schedule),
        'total_cost': round(total_cost, 2)
    }

# Calculate all possible plans
costs = []
for day in DAYS:
    for car_id in CAR_IDS:
        for hour in range(0, 24):
            plan = calculate_cost(car_id, day, hour, 0, availability, prices, cars, efficiency=EFFICIENCY)
            if plan:
                costs.append(plan)

costs_df = pd.DataFrame(costs)

# Select fair plans and enforce max port constraint
costs_df.sort_values('total_cost', inplace=True)
selected_plans = []
used_ports = defaultdict(int)
assigned = defaultdict(set)  # car_id → set of assigned days

for _, plan in costs_df.iterrows():
    car_id, day, start, finish = plan['car_id'], plan['day'], plan['start_hour'], plan['finish_hour']

    if day in assigned[car_id]:
        continue

    if all(used_ports[h] < MAX_PORTS for h in range(start, finish + 1)):
        selected_plans.append(plan)
        for h in range(start, finish + 1):
            used_ports[h] += 1
        assigned[car_id].add(day)

# Ensure last hour charging is assigned to one of the cars (to keep the cost low)
def ensure_last_hour_charging(plans, days):
    modified_plans = plans.copy()

    for day in days:
        last_hour = 23 + int(day) * 24
        if not any(plan['day'] == day and plan['start_hour'] == last_hour for plan in modified_plans):
            for plan in modified_plans:
                if plan['day'] == day and plan['start_hour'] < last_hour:
                    plan['start_hour'] = last_hour
                    plan['finish_hour'] = last_hour + (plan['finish_hour'] - plan['start_hour'])
                    break
    return modified_plans

In [5]:
selected_plans_with_last_hour = ensure_last_hour_charging(selected_plans, DAYS)

In [6]:
# Convert to DataFrame
fair_plan_df = pd.DataFrame(selected_plans_with_last_hour)
fair_plan_df['start_hour_in_day'] = fair_plan_df['start_hour'] % 24
fair_plan_df['finish_hour_in_day'] = fair_plan_df['finish_hour'] % 24

# Evaluate fairness
total_costs_per_car = fair_plan_df.groupby('car_id')['total_cost'].sum()
fairness_std = np.std(total_costs_per_car)

In [7]:
print("\nDaily schedule:")
print(fair_plan_df.sort_values(by=['day', 'car_id']))


Daily schedule:
   car_id day  start_hour  finish_hour  hours_charging  total_cost  \
0   Car_1   0          18           20               3       23.58   
7   Car_2   0          21           23               3       16.84   
12  Car_3   0          23           23               2       13.89   
16  Car_4   0          21           23               3       23.58   
17  Car_1   1          42           44               3       23.58   
24  Car_2   1          47           47               3       16.84   
26  Car_3   1          43           45               3       23.16   
30  Car_4   1          43           46               4       29.47   
32  Car_1   2          66           68               3       23.58   
39  Car_2   2          69           71               3       16.84   
44  Car_3   2          71           71               2       13.89   
47  Car_4   2          68           70               3       23.58   
49  Car_1   3          90           92               3       23.58   
56 

In [8]:
# Final SoC calculation and cost
def calculate_final_soc(initial_soc, charging_schedule, car_id):
    car = next(car for car in cars if car['id'] == car_id)
    total_energy_charged = sum(charging_schedule.values())
    final_soc = initial_soc + (total_energy_charged / car['capacity_kWh'])
    return round(min(final_soc, 1.0), 2)

for idx, plan in fair_plan_df.iterrows():
    initial_soc = 0  # Assuming initial SoC is 0
    charging_schedule = {h: plan['total_cost'] / (prices.loc[h, 'price_eur_per_kWh'] * 0.95) for h in range(plan['start_hour'], plan['finish_hour'] + 1)}
    final_soc = calculate_final_soc(initial_soc, charging_schedule, plan['car_id'])
    fair_plan_df.at[idx, 'final_soc'] = final_soc

print("\nCharging Schedule and Final SoC:")
for idx, plan in fair_plan_df.iterrows():
    print(f"\nCar: {plan['car_id']} (Day: {plan['day']})")
    print(f"Total Cost: {plan['total_cost']} EUR")
    print(f"Final SoC: {plan['final_soc']}")

def calculate_energy_charged(fair_plan_df):
    energy_charged_summary = defaultdict(float)
    for idx, plan in fair_plan_df.iterrows():
        charging_schedule = {h: plan['total_cost'] / (prices.loc[h, 'price_eur_per_kWh'] * 0.95) for h in range(plan['start_hour'], plan['finish_hour'] + 1)}
        total_energy_charged = sum(charging_schedule.values())
        energy_charged_summary[plan['car_id']] += total_energy_charged

    print("\nTotal Energy Charged (kWh) per Car:")
    for car_id, energy_charged in energy_charged_summary.items():
        print(f"{car_id}: {round(energy_charged, 2)} kWh")


Charging Schedule and Final SoC:

Car: Car_3 (Day: 4)
Total Cost: 13.89 EUR
Final SoC: 0.0

Car: Car_3 (Day: 2)
Total Cost: 13.89 EUR
Final SoC: 0.66

Car: Car_3 (Day: 0)
Total Cost: 13.89 EUR
Final SoC: 0.66

Car: Car_2 (Day: 1)
Total Cost: 16.84 EUR
Final SoC: 0.89

Car: Car_2 (Day: 2)
Total Cost: 16.84 EUR
Final SoC: 1.0

Car: Car_2 (Day: 3)
Total Cost: 16.84 EUR
Final SoC: 0.89

Car: Car_2 (Day: 4)
Total Cost: 16.84 EUR
Final SoC: 1.0

Car: Car_2 (Day: 5)
Total Cost: 16.84 EUR
Final SoC: 0.0

Car: Car_2 (Day: 0)
Total Cost: 16.84 EUR
Final SoC: 1.0

Car: Car_3 (Day: 1)
Total Cost: 23.16 EUR
Final SoC: 1.0

Car: Car_3 (Day: 3)
Total Cost: 23.16 EUR
Final SoC: 1.0

Car: Car_3 (Day: 5)
Total Cost: 23.16 EUR
Final SoC: 1.0

Car: Car_1 (Day: 4)
Total Cost: 23.58 EUR
Final SoC: 1.0

Car: Car_1 (Day: 0)
Total Cost: 23.58 EUR
Final SoC: 1.0

Car: Car_4 (Day: 4)
Total Cost: 23.58 EUR
Final SoC: 1.0

Car: Car_1 (Day: 5)
Total Cost: 23.58 EUR
Final SoC: 1.0

Car: Car_4 (Day: 2)
Total Cost: 2

In [9]:
# Call the function to for the energy charged summary
calculate_energy_charged(fair_plan_df)


Total Energy Charged (kWh) per Car:
Car_3: 621.63 kWh
Car_2: 487.47 kWh
Car_1: 1116.95 kWh
Car_4: 1178.89 kWh


In [10]:
weekly_costs = {}

# Iterate through the DataFrame to accumulate the total costs for each car
for _, plan in fair_plan_df.iterrows():
    car_id = plan['car_id']
    daily_cost = plan['total_cost']

    if car_id not in weekly_costs:
        weekly_costs[car_id] = 0

    weekly_costs[car_id] += daily_cost

for car_id, total_cost in weekly_costs.items():
    print(f"Total weekly cost for Car {car_id}: {total_cost:.2f} EUR")

Total weekly cost for Car Car_3: 111.15 EUR
Total weekly cost for Car Car_2: 101.04 EUR
Total weekly cost for Car Car_1: 141.48 EUR
Total weekly cost for Car Car_4: 129.68 EUR
